# Primeros pasos con Spark

In [ ]:
%%configure 
{"executorMemory": "3072M", "executorCores": 4, "numExecutors":10}


In [ ]:
# las siguientes tres líneas importan e inicializan el contexto Spark. Para un notebook no es necesario puesto que la
# primera ejecución de un comando ya inicializa los contextos

from pyspark import SparkConf, SparkContext
#cnf = SparkConf().setMaster("local").setAppName("WordCount")
#sc = SparkContext(conf = cnf)

# usando Spark para contar palabras

txt = sc.textFile("wasb:///example/data/gutenberg/ulysses.txt")
words = txt.flatMap(lambda txt: txt.split(" "))
counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
# store results
counts.saveAsTextFile("/wordcount_output")

# Consultar si ha creado el fichero desde el shell 
# Borramos el directorio /wordcount_output para siguientes ejecuciones

# # DataFrames
Desde Spark 2.0 los contextos de HiveContext y SQLContext están integrados en SparkContext que es el mecanismo recomendado para utilizar Spark SQL a día de hoy

En las siguientes lineas de código vemos como importar ficheros, primero de texto, después csv para crear DataFrames a partir de ellos

In [ ]:
csv = spark.read.text('wasb:///HdiSamples/HdiSamples/SensorSampleData/building/building.csv')
csv.printSchema()

In [ ]:
csv.show(truncate = False)

In [ ]:
building_csv = spark.read.csv('wasb:///HdiSamples/HdiSamples/SensorSampleData/building/building.csv', header=True, inferSchema=True)
building_csv.printSchema()

In [ ]:
building_csv.show()

Ahora generamos una estructura que llamaremos schma con el esquema de HVAC.csv para cargar adecuadamente los datos

In [ ]:
from pyspark.sql.types import *
schma = StructType([
StructField("Date", StringType(), False),
StructField("Time", StringType(), False),
StructField("TargetTemp", IntegerType(), False),
StructField("ActualTemp", StringType(), False),
StructField("System", IntegerType(), False),
StructField("SystemAge", IntegerType(), False),
StructField("BuildingID", IntegerType(), False),
])
#Leemos el csv infomando el esquema e indicando que el fichero tiene cabecera
hvac_csv = spark.read.csv('wasb:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv', schema=schma, header=True)
hvac_csv.printSchema()

In [ ]:
#Mostramos los datos cargados

hvac_csv.show()

In [ ]:
# generamos otro DataFrame building_data y hvac_Data filtrando tanto columnas como filas a modo de ejemplo

building_data = building_csv.select("BuildingID", "BuildingAge", "HVACproduct")
building_data.show()

from pyspark.sql.functions import *

hvac_data = hvac_csv.select("BuildingID", "ActualTemp", "TargetTemp").filter(col("ActualTemp") > col("TargetTemp"))
hvac_data.show()

In [ ]:
#Hacemos un join entre las dos tablas 
hot_buildings = building_data.join(hvac_data, "BuildingID")
hot_buildings.show()

Uso de SQL para consulta de tablas. Cremos una vista temporal para poder consultarla con sentencias SELECT

In [ ]:
hot_buildings.createOrReplaceTempView("tmpHotBuildings")


In [ ]:
%%sql
SELECT HVACProduct, AVG(ActualTemp - TargetTemp) AS AvgError
FROM tmpHotBuildings
GROUP BY HVACproduct
ORDER BY HVACproduct

In [ ]:
building_csv.write.saveAsTable("buildingSpark")
building_df = spark.sql("SELECT * FROM buildingSpark")
building_df.show()

Consultar una tabla HIVE

In [ ]:
calls = spark.sql("""SELECT devicemodel, COUNT(*) AS calls
FROM hivesampletable
GROUP BY devicemodel
ORDER BY calls DESC """)
calls.show()